In [5]:
import copy
import nilearn.image as nimg
import pandas as pd
from confounds import censor_method
import importlib
import nibabel as nib
import numpy as np

In [2]:
base_cols = ["white_matter","csf","global_signal"]
sq = True
deriv = True
sqderiv = True

all_cols = copy.copy(base_cols)
if sq:
    all_cols += [f"{f}_power2" for f in base_cols]
    
if deriv:
    all_cols += [f"{f}_derivative1" for f in base_cols]
    
if sqderiv:
    all_cols += [f"{f}_derivative1_power2" for f in base_cols]
    
clean_dict = {
    "--cf-cols": ','.join(all_cols),
    "detrend": True, "standardize": True,
    "low_pass": 0.08,
    "high_pass": 0.009
}

## Testing Utility Functions

`_get_vol_index`

In [10]:
data = np.zeros((10,10,10,10))
nifti = nib.Nifti1Image(data, np.eye(4))
indices = np.array([0,1,2,3])
res = censor_method._get_vol_index(nifti, indices)

assert res.shape[-1] == len(indices)

`_image_to_signals`

In [11]:
data = np.zeros((10, 10, 10, 10))
nifti = nib.Nifti1Image(data, np.eye(4))
res = censor_method._image_to_signals(nifti)
expected_dim = (10*10*10, 10)
assert res.shape == expected_dim

`_clear_steady_state` when `confounds` passed

In [29]:
data = np.zeros((10, 10, 10, 10))
nifti = nib.Nifti1Image(data, np.eye(4))

# Construct steady_state dummy columns
steady_state = pd.Series(np.arange(10) + 1)
steady_state[5:] = 0
confounds = pd.DataFrame(steady_state, columns=["steady"])
confounds = pd.get_dummies(confounds, columns=["steady"])
confounds.drop('steady_0', inplace=True, axis=1)

res_img, res_conf  = censor_method._clear_steady_state(nifti, confounds)


assert res_img.shape[-1] == 5
assert res_conf.shape[0] == 5

`_clear_steady_state` when `drop_trs` passed

In [30]:
data = np.zeros((10, 10, 10, 10))
nifti = nib.Nifti1Image(data, np.eye(4))

# Construct steady_state dummy columns
steady_state = pd.Series(np.arange(10) + 1)
steady_state[5:] = 0
confounds = pd.DataFrame(steady_state, columns=["steady"])
confounds = pd.get_dummies(confounds, columns=["steady"])
confounds.drop('steady_0', inplace=True, axis=1)

res_img, res_conf = censor_method._clear_steady_state(nifti, confounds,
                                                     drop_trs=2)

assert res_img.shape[-1] == 8
assert res_conf.shape[0] == 8

`_clear_steady_state` fails when steady columns not supplied

In [34]:
data = np.zeros((10, 10, 10, 10))
nifti = nib.Nifti1Image(data, np.eye(4))

# Construct steady_state dummy columns
steady_state = pd.Series(np.arange(10) + 1)
steady_state[5:] = 0
confounds = pd.DataFrame(steady_state, columns=["a"])
confounds = pd.get_dummies(confounds, columns=["a"])

res_img, res_conf = censor_method._clear_steady_state(nifti, confounds)

04/07/2021 12:57:31 PM drop_trs not supplied and steady_state volumes not found in confounds file!


ValueError: 

`_interpolate_frames` returns Nifti1Image of correct size when run

In [44]:
data = np.random.uniform(size=(10, 10, 10, 10))
nifti = nib.Nifti1Image(data, np.eye(4))

mask = np.array([0,1,2,3,7,8,9])
censor = np.array([4,5,6])

res_img = censor_method._interpolate_frames(nifti, mask, censor, t_r=2)

assert res_img.shape == nifti.shape
assert isinstance(res_img, nib.Nifti1Image)

`_interpolate_frames` returns np.ndarray of correct size when run

In [47]:
data = np.random.uniform(size=(10*10*10, 10))

mask = np.array([0,1,2,3,7,8,9])
censor = np.array([4,5,6])

res_img = censor_method._interpolate_frames(data, mask, censor, t_r=2)

assert res_img.shape == data.shape
assert isinstance(res_img, np.ndarray)

`_interpolate_frames` leaves masked values unchanged

In [49]:
data = np.random.uniform(size=(10*10*10, 10))

mask = np.array([0,1,2,3,7,8,9])
censor = np.array([4,5,6])

res_img = censor_method._interpolate_frames(data, mask, censor, t_r=2)

assert np.array_equal(res_img[:, mask], data[:, mask])